curl -sS 'http://localhost:5000/config?name=ohad&user_id=1'

In [1]:
# dir(protobuf3)

In [22]:
import asd_pb2
import gzip
# import protobuf3
import struct
# print(dir(asd_pb2._USER))
# print("\n".join(dir(asd_pb2.User)))
class Reader:
    def __init__(self, path="sample.mind.gz"):
        self._path  = path
        self._read_user()
        
    def _read_user(self):
        with gzip.open(self._path) as f:
            user_msg_size = struct.unpack("I",f.read(4))[0]
            user_msg_bytes = f.read(user_msg_size)
            user = asd_pb2.User.FromString(user_msg_bytes)
            self.user_id = user.user_id
            self.username = user.username
            self.birthday = user.birthday
                
    def __iter__(self):
        with gzip.open(self._path) as f:
            user_msg_size = struct.unpack("I",f.read(4))[0]
            user_msg_bytes = f.read(user_msg_size)
            while True:
                msg_size = struct.unpack("I",f.read(4))[0]
                msg_bytes = f.read(msg_size)
                snapshot = asd_pb2.Snapshot.FromString(msg_bytes)
                yield snapshot


In [26]:
import inspect


import requests
from reader import Reader
import asd_pb2
addr = "http://localhost:5000"
reader = Reader()
user_fields = {"user_id":reader.user_id,"username":reader.username,
               "birthday":reader.birthday,"gender":reader.gender}
user = asd_pb2.User(**user_fields)

for i,snapshot in enumerate(reader):
    resp_result = requests.get(f'{addr}/config',data=user_fields).json()
    available_parsers = resp_result['parsers']
    snapshot_fields = {"color_image":snapshot.color_image,
                       "pose":snapshot.pose,
                       "depth_image":snapshot.depth_image,
                       "feelings":snapshot.feelings} 
    
    snapshot = asd_pb2.Snapshot(**{parser_name:snapshot_fields[parser_name] 
                                   for parser_name in available_parsers})
    packet = asd_pb2.Packet(snapshot=snapshot,user=user)
    resp = requests.post(f'{addr}/config',data=packet.SerializeToString(),
                         headers={'Content-Type': 'application/protobuf',
                                  'Content-Length':  str(len(packet_b))})
#     print(resp.json())
#     print(a.depth_image.ByteSize())
    if i==5:
        break



In [18]:
r

{'data': {'user_id': 1, 'name': 'ohad', 'birthday': None, 'gender': None},
 'parsers': ['pose', 'color_image', 'depth_image', 'feelings']}